# ML2025 Homework 1 - Retrieval Augmented Generation with Agents

## Environment Setup

First, we will mount your own Google Drive and change the working directory.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
# Change the working directory to somewhere in your Google Drive.
# You could check the path by right clicking on the folder.
%cd /content/drive/MyDrive/ML/hw1


/content/drive/MyDrive/ML/hw1


In this section, we install the necessary python packages and download model weights of the quantized version of LLaMA 3.1 8B. Also, download the dataset. Note that the model weight is around 8GB. If you are using your Google Drive as the working directory, make sure you have enough space for the model.

In [20]:
!python3 -m pip install --no-cache-dir llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
!python3 -m pip install googlesearch-python bs4 charset-normalizer requests-html lxml_html_clean

from pathlib import Path
if not Path('./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf').exists():
    !wget https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf
if not Path('./public.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/public.txt
if not Path('./private.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/private.txt

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122


In [21]:
import torch
if not torch.cuda.is_available():
    raise Exception('You are not using the GPU runtime. Change it first or you will suffer from the super slow inference speed!')
else:
    print('You are good to go!')

You are good to go!


## Prepare the LLM and LLM utility function

By default, we will use the quantized version of LLaMA 3.1 8B. you can get full marks on this homework by using the provided LLM and LLM utility function. You can also try out different LLM models.

In the following code block, we will load the downloaded LLM model weights onto the GPU first.
Then, we implemented the generate_response() function so that you can get the generated response from the LLM model more easily.

You can ignore "llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized" warning.

In [22]:
from llama_cpp import Llama

# Load the model onto GPU
llama3 = Llama(
    "./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf",
    verbose=False,
    n_gpu_layers=-1,
    n_ctx=16384,    # This argument is how many tokens the model can take. The longer the better, but it will consume more memory. 16384 is a proper value for a GPU with 16GB VRAM.
)

def generate_response(_model: Llama, _messages: str) -> str:
    '''
    This function will inference the model with given messages.
    '''
    _output = _model.create_chat_completion(
        _messages,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        max_tokens=512,    # This argument is how many tokens the model can generate, you can change it and observe the differences.
        temperature=0,      # This argument is the randomness of the model. 0 means no randomness. You will get the same result with the same input every time. You can try to set it to different values.
        repeat_penalty=2.0,
    )["choices"][0]["message"]["content"]
    return _output

ValueError: Failed to load model from file: ./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf

## Search Tool

The TA has implemented a search tool for you to search certain keywords using Google Search. You can use this tool to search for the relevant **web pages** for the given question. The search tool can be integrated in the following sections.

In [6]:
from typing import List
from googlesearch import search as _search
from bs4 import BeautifulSoup
from charset_normalizer import detect
import asyncio
from requests_html import AsyncHTMLSession
import urllib3
urllib3.disable_warnings()

async def worker(s:AsyncHTMLSession, url:str):
    try:
        header_response = await asyncio.wait_for(s.head(url, verify=False), timeout=10)
        if 'text/html' not in header_response.headers.get('Content-Type', ''):
            return None
        r = await asyncio.wait_for(s.get(url, verify=False), timeout=10)
        return r.text
    except:
        return None

async def get_htmls(urls):
    session = AsyncHTMLSession()
    tasks = (worker(session, url) for url in urls)
    return await asyncio.gather(*tasks)

async def search(keyword: str, n_results: int=3) -> List[str]:
    '''
    This function will search the keyword and return the text content in the first n_results web pages.

    Warning: You may suffer from HTTP 429 errors if you search too many times in a period of time. This is unavoidable and you should take your own risk if you want to try search more results at once.
    The rate limit is not explicitly announced by Google, hence there's not much we can do except for changing the IP or wait until Google unban you (we don't know how long the penalty will last either).
    '''
    keyword = keyword[:100]
    # First, search the keyword and get the results. Also, get 2 times more results in case some of them are invalid.
    results = list(_search(keyword, n_results * 2, lang="zh", unique=True))
    # Then, get the HTML from the results. Also, the helper function will filter out the non-HTML urls.
    results = await get_htmls(results)
    # Filter out the None values.
    results = [x for x in results if x is not None]
    # Parse the HTML.
    results = [BeautifulSoup(x, 'html.parser') for x in results]
    # Get the text from the HTML and remove the spaces. Also, filter out the non-utf-8 encoding.
    results = [''.join(x.get_text().split()) for x in results if detect(x.encode()).get('encoding') == 'utf-8']
    # Return the first n results.
    return results[:n_results]

## Test the LLM inference pipeline

In [ ]:
# You can try out different questions here.
test_question='請問誰是 Taylor Swift？'

messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。"},    # System prompt
    {"role": "user", "content": test_question}, # User prompt
]

print(generate_response(llama3, messages))

泰勒絲（Taylor Swift）是一位美國歌手、詞曲作家和音樂製作人。她出生於1989年，來自田納西州。她的音乐风格从乡村摇滚发展到流行搖擺，並且她被誉为当代最成功的女艺人的之一。

泰勒絲早期在鄉郊小鎮演唱會時開始發展音樂事業，她推出了多張專輯，包括《Taylor Swift》、《Fearless》，以及後來更為知名的大熱作如 《1989》（2014年）、_reputation（）和 _Lover （）。她的歌曲經常探討愛情、友誼及自我成長等主題。

泰勒絲獲得了許多獎項，包括13座格萊美奖，並且是史上最快達到百萬銷量的女藝人之一。


## Agents

The TA has implemented the Agent class for you. You can use this class to create agents that can interact with the LLM model. The Agent class has the following attributes and methods:
- Attributes:
    - role_description: The role of the agent. For example, if you want this agent to be a history expert, you can set the role_description to "You are a history expert. You will only answer questions based on what really happened in the past. Do not generate any answer if you don't have reliable sources.".
    - task_description: The task of the agent. For example, if you want this agent to answer questions only in yes/no, you can set the task_description to "Please answer the following question in yes/no. Explanations are not needed."
    - llm: Just an indicator of the LLM model used by the agent.
- Method:
    - inference: This method takes a message as input and returns the generated response from the LLM model. The message will first be formatted into proper input for the LLM model. (This is where you can set some global instructions like "Please speak in a polite manner" or "Please provide a detailed explanation".) The generated response will be returned as the output.

In [10]:
class LLMAgent():
    def __init__(self, role_description: str, task_description: str, llm:str="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF"):
        self.role_description = role_description   # Role means who this agent should act like. e.g. the history expert, the manager......
        self.task_description = task_description    # Task description instructs what task should this agent solve.
        self.llm = llm  # LLM indicates which LLM backend this agent is using.
    def inference(self, message:str) -> str:
        if self.llm == 'bartowski/Meta-Llama-3.1-8B-Instruct-GGUF': # If using the default one.
            # TODO: Design the system prompt and user prompt here.
            # Format the messsages first.
            messages = [
                {"role": "system", "content": f"{self.role_description}"},  # Hint: you may want the agents to speak Traditional Chinese only.
                {"role": "user", "content": f"{self.task_description}\n{message}"}, # Hint: you may want the agents to clearly distinguish the task descriptions and the user messages. A proper seperation text rather than a simple line break is recommended.
            ]
            return generate_response(llama3, messages)
        else:
            # TODO: If you want to use LLMs other than the given one, please implement the inference part on your own.
            return ""

TODO: Design the role description and task description for each agent.

In [36]:
# TODO: Design the role and task description for each agent.

# This agent may help you filter out the irrelevant parts in question descriptions.
question_extraction_agent = LLMAgent(
    role_description="你是 LLaMa-3.1-8B，是用來擷取問題的 AI。使用中文時只會使用繁體中文來回問題。你要將收到的問題中不重要的資訊去除，除了題目中的問句以外，你要確保題幹中相關的條件和資訊都沒有喪失。你不要根據你自己的知識回答問題或是加入任何資訊。你的回答應該是一個可以和原本問題得到相同答案的簡潔問句。你應該要著重於問題的精髓，不要被無關的資訊影響，例如，如果原本的問題是「小明本來有五顆蘋果，他吃掉了一顆，又送給朋友一顆，請問光速是多少」，你應該要知道「小明」和「蘋果」是不重要的資訊，重要的問題是「光速是多少」。例如，如果原本的問題是「細胞是構成生物的基本單位，關於細胞的研究已經為人類在生命科學領域帶來了極大的啟發，至今仍有重大的突破。請問細胞的主要成份為何？」，你應該回答簡化過後的問題「請問細胞的主要成份為何？」",
    task_description="請將收到的問題中與問題本身無關的資訊和內容去除，並在不改變原本問題意思的情況下提供盡量簡潔的問句，你的回答應該只含有問句。",
)

# This agent may help you extract the keywords in a question so that the search tool can find more accurate results.
keyword_extraction_agent = LLMAgent(
    role_description="你是 LLaMa-3.1-8B，是用來擷取關鍵字的 AI。根據收到的問題擷取出關鍵字，關鍵字應該是原來的問題中就有的詞彙。使用中文時只會使用繁體中文來回問題。回答問題時請將不同關鍵字以一個空格分開。不要根據你的知識回答問題，不要在關鍵字中加入原本的問題中沒有的資訊，只要分析問題並提出關鍵字就好。如果問題是給你一些資訊，要你找出相關的主題是什麼，那你的關鍵字應該要包含這些資訊。例如，如果原本的問題是「請問斯是陋室、惟吾德馨是誰說的」，你的關鍵字應該要包含「斯是陋室，惟吾德馨」你要確保原本的問題中的資訊和條件都被包含在關鍵字裡面。關鍵字都應該要是原本的問題中就有的詞彙。例如，如果原本的問題是「三菱集團的創始人是誰」，你應該擷取出關鍵字「三菱集團 創始人」。如果原本的問題是「nba 1999賽季得分王是誰」，你應該擷取出關鍵字「nba 1999 得分王」",
    task_description="請根據收到的問題擷取出方便進行網路搜尋的關鍵字，在不損失問題含義的情況下越精簡越好。",
)

# This agent is the core component that answers the question.
qa_agent = LLMAgent(
    role_description="你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題，並且使用在台灣地區常見的說法。你只要針對問題給出答案就好，不用回答多餘的內容。請確保通順地回答原本的問題，在這個條件下答案可以盡量精簡。請避免問題和答案間有不通順的情況發生。",
    task_description="請根據你的知識和網路搜尋的結果，給出對以下問題最合理的答案。",
)

# This agent only gives answer based on the knowledge of the LLM
truth_agent = LLMAgent(
  role_description="你是 LLaMA-3.1-8B，是用來提供資訊的 AI。你只會根據你所知道的事實回答問題，如果不知道就要說「我不知道」，你的回答只需要含有問題的答案。使用中文時只會使用繁體中文來回問題。",
  task_description="請根據以下問題，根據你知道的事實回答。回答要符合指定的格式。"
)

## RAG pipeline

TODO: Implement the RAG pipeline.

Please refer to the homework description slides for hints.

Also, there might be more heuristics (e.g. classifying the questions based on their lengths, determining if the question need a search or not, reconfirm the answer before returning it to the user......) that are not shown in the flow charts. You can use your creativity to come up with a better solution!

- Naive approach (simple baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive.png)

- Naive RAG approach (medium baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive_rag.png)

- RAG with agents (strong baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/rag_agent.png)

In [33]:
async def pipeline(question: str) -> str:
    # TODO: Implement your pipeline.
    # Currently, it only feeds the question directly to the LLM.
    # You may want to get the final results through multiple inferences.
    # Just a quick reminder, make sure your input length is within the limit of the model context window (16384 tokens), you may want to truncate some excessive texts.
    # truth_agent_response: str = truth_agent.inference(concise_question);
    if len(question) > 50:
      concise_question: str = question_extraction_agent.inference(question);
      keywords: str = keyword_extraction_agent.inference(concise_question);
    elif len(question) > 30:
      keywords: str = keyword_extraction_agent.inference(question);
    else:
      keywords: str = question;
    # keywords: str = keyword_extraction_agent.inference(concise_question);
    print(f"keywords: {keywords}");
    search_result: List[str] = await search(keyword=keywords, n_results=3);
    combined_search_result: str = "";
    for result in search_result:
      combined_search_result = combined_search_result + result[0:4000] + " ";
    if len(combined_search_result) > 12000:
      combined_search_result = combined_search_result[0:12000];
    print(f"serach result: {combined_search_result}");
    qa_prompt:str = f"問題：{question},  搜尋結果：{combined_search_result} "
    print(f"final promp: {qa_prompt}");
    return qa_agent.inference(qa_prompt)

## Answer the questions using your pipeline!

In [37]:
%rm b12901022*

Since Colab has usage limit, you might encounter the disconnections. The following code will save your answer for each question. If you have mounted your Google Drive as instructed, you can just rerun the whole notebook to continue your process.

In [38]:
from pathlib import Path

# Fill in your student ID first.
STUDENT_ID = "B12901022"

STUDENT_ID = STUDENT_ID.lower()
with open('./public.txt', 'r') as input_f:
    questions = input_f.readlines()
    questions = [l.strip().split(',')[0] for l in questions]
    for id, question in enumerate(questions, 1):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            print("fo")
            continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'w') as output_f:
            print(answer, file=output_f)

with open('./private.txt', 'r') as input_f:
    questions = input_f.readlines()
    for id, question in enumerate(questions, 31):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'a') as output_f:
            print(answer, file=output_f)

keywords: 虎山雄風飛揚 校歌
serach result: 校歌南投縣南投市光華國小-精華區lyrics-批踢踢實業坊批踢踢實業坊›精華區betalyrics關於我們聯絡資訊返回上層作者ypx0409(ypx0409)看板lyrics標題校歌南投縣南投市光華國小時間TueSep1603:13:342008貓羅溪旁虎山雄風飛揚這兒是我們生長的地方良師益友濟濟一堂克勤克儉奮發圖強長我育我飲水思源立定志向四海名揚忠心復國是理想光華意志堅光華氣勢昂你我相共勉誓把華夏重光--※發信站:批踢踢實業坊(ptt.cc)◆From:122.116.179.102推vibrant224:國小校歌推...09/1611:10推sognare:校友友情推09/1614:09 走過一甲子跳到主要內容區您的瀏覽器不支援JavaScript功能，若網頁功能無法正常使用時，請開啟瀏覽器JavaScript狀態Menu回首頁Home網站導覽WebGuide關於光華AboutUs學校簡介校史沿革學校概況光華校旗光華校歌光華校景交通位置光華學生數光華一甲子60GloriousYears歷屆校長光華一甲子再創百年史走過一甲子資優教育史體育輝煌史音樂璀璨史藝文榮耀史光華校園誌歷史剪影60週年活動花絮行政處室OurTeam校長室教務處學務處輔導處總務處人事室會計室校務應用RelatedWebs校務行政系統南投縣教育處教師在職進修網WebITR差勤系統課務排代數位學習推動辦公室校園雲端電子信箱學習扶助科技評量啟用教育雲端帳號(OpenID)校園午餐登錄平台各類獎狀列印畢業獎項列印公文整合資訊系統會計系統校園花絮Photos電子相簿線上影音校內網站websites光華FB粉絲頁班級網站光華資優班特教研習及活動資訊不分類資源班光華交通安全網光華圖書室光華附設幼兒園南投縣資優資源教育中心校園導覽VR720自主學習NeverStopLearning校園學生資源網googleclassroom登入GoogleClassroom登入教學教育體系單一簽入服務教育部因材網PaGamO線上學習平台線上教學便利包康軒雲南一OneBox翰林行動大師真平本土語在家學何嘉仁ABC英語世界學習吧均一教育平台CoolEnglish線上學習平台Hami書城-校園電子書平台瓜瓜漂流網閱讀認證學校行事曆Calendar公告管理Manag

In [39]:
# Combine the results into one file.
with open(f'./{STUDENT_ID}.txt', 'w') as output_f:
    for id in range(1,91):
        with open(f'./{STUDENT_ID}_{id}.txt', 'r') as input_f:
            answer = input_f.readline().strip()
            print(answer, file=output_f)
            print(answer)

「虎山雄風飛揚」是南投縣光華國小的校歌。
根據NCC的公告，自2025年2月14日起暫緩收取《第二級電信管制射頻器材輸入核准審查費》，即750元。
第一代 iPhone 是由史蒂夫·乔布斯發表。
根據提供的資訊，托福網路測驗 TOEFL iBT 要達到多少分才能申請台灣大學進階英文免修尚未明確，但一般而言，大多數學校都會要求考生取得 80 分以上。
觸地得分（達陣Try）可得到5個點。
ruvuwa'an位於現今的太麻里鄉。
熊仔的碩班指導教授為李琳山。
法拉第是電磁學的先驅，他發現了电动势与通过一闭合曲线（如环形导体）的变化率之间存在着密切关系。
康樂車站
20+30=50
達拉斯獨行俠隊的當家球星Luka Doncic被交易至洛杉磯湖人。
2024年美國總統選舉的勝出者是唐納·川普。
0.02B
依據國立臺灣大學學則，沒有申請書的情況下，每個在校生每年最多停修3門課程。
DeepSeek的母公司是幻方量化。
2024年NBA总冠军是波士顿凯尔特人队。
烴類化合物中，碳原子與其他非氫元素形成鍵的稱為「有機功能團」，而不是三键。
艾伦·图灵是被誉为「計算機科學之父」的英国电脑科学家、数学学者和逻辑思想家的。他提出了著名的圖靈机概念，为现代计算理論奠定了基础。
南投縣名間鄉
Windows 作業系統是微軟公司的產品。
官將首起源自三峽總院區。
《咒》的邪神名為「阿修羅」。
短暫交會的旅程就此分岔是動力火車歌曲路人甲中的詞句。
2025年卑南族聯合年的活動將由利嘉部落主辦。
GeForce RTX 50系列
大S是在日本旅遊時因病去世。
牛頓發現萬有引力定律後，等了22年才對全世界公開。這可能是因為他根據月亮與地球的距離比較重力的計算結果，不符合平方反比定的理論。他當時估算的地球和太陽之間相距約1.5億公里，但實際上只有76萬多千米，差異很大，這使得他的研究方向轉移到其他科學問題上了。
TAIHUCAIS
“我会回来的”是出自1984年阿诺·施瓦辛格的科幻电影《终结者》的经典台词。
水的化學式為H2O。
第15個作業的名稱是：為什麼語言模型用文字接龍，圖片生成不像素連續呢？—淺談產生式人工智慧中的發散策略
目前臺灣多數獨立學院皆已升格為大學，公 立的唯一一間是馬偕醫學專科學校
BitTorrent 协议采用了优化的不 choking 机制来确保新加入网络且尚无任何 ch